# Market Basket Analysis

This is my market basket analysis project on a Indian E-Commerce dataset. This project identifies hidden trends in customer purchasing behavior by analyzing what products are frequently bought together in a E-Commerce's transaction. Market basket analysis is one of the data analysis techniques that aims to identify frequently occurring itemsets or combinations of products to understand customer buying habits.

You can get the dataset also on [Kaggle E-Commerce Dataset](https://www.kaggle.com/datasets/benroshan/ecommerce-data)!

In [57]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

## Load Dataset

In [58]:
df = pd.read_csv('Dataset/Order Details.csv')
df

,Order ID,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,66.0,-12.0,5,Clothing,Stole
2,B-25601,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,168.0,-111.0,2,Electronics,Phones
...,...,...,...,...,...,...
1495,B-26099,835.0,267.0,5,Electronics,Phones
1496,B-26099,2366.0,552.0,5,Clothing,Trousers
1497,B-26100,828.0,230.0,2,Furniture,Chairs
1498,B-26100,34.0,10.0,2,Clothing,T-shirt


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order ID      1500 non-null   object 
 1   Amount        1500 non-null   float64
 2   Profit        1500 non-null   float64
 3   Quantity      1500 non-null   int64  
 4   Category      1500 non-null   object 
 5   Sub-Category  1500 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 70.4+ KB


In [60]:
df.describe()

,Amount,Profit,Quantity
count,1500.000000,1500.000000,1500.000000
mean,287.668000,15.970000,3.743333
std,461.050488,169.140565,2.184942
min,4.000000,-1981.000000,1.000000
25%,45.000000,-9.250000,2.000000
50%,118.000000,9.000000,3.000000
75%,322.000000,38.000000,5.000000
max,5729.000000,1698.000000,14.000000


In [61]:
df['Sub-Category'].value_counts()

Sub-Category
Saree               210
Hankerchief         198
Stole               192
Phones               83
Bookcases            79
Electronic Games     79
T-shirt              77
Printers             74
Chairs               74
Furnishings          73
Accessories          72
Shirt                69
Skirt                64
Leggings             53
Kurti                47
Trousers             39
Tables               17
Name: count, dtype: int64

## Create Basket

In [69]:
basket_df = df.groupby('Order ID')['Sub-Category'].agg(set).reset_index()
basket_df['Sub-Category'] = basket_df['Sub-Category'].apply(lambda x: '|'.join(x))
basket_df = basket_df['Sub-Category'].str.get_dummies('|').astype('bool')
basket_df

,Accessories,Bookcases,Chairs,Electronic Games,Furnishings,Hankerchief,Kurti,Leggings,Phones,Printers,Saree,Shirt,Skirt,Stole,T-shirt,Tables,Trousers
0,False,True,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False
2,False,False,True,False,False,True,True,False,False,False,True,False,False,True,False,False,True
3,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,True,False,False,False,False,False,False,True,True,False,True,True,False,False,False,False,True
496,False,False,True,True,False,True,False,True,False,True,False,False,False,False,False,False,False
497,True,False,True,False,False,False,False,True,False,False,True,False,True,False,True,False,False
498,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,True


## Create Frequent Itemsets

In [82]:
frequent_itemsets = apriori(basket_df,min_support=0.01,use_colnames=True).sort_values('support',ascending=False).reset_index(drop=True)
frequent_itemsets

,support,itemsets
0,0.314,(Stole)
1,0.312,(Saree)
2,0.276,(Hankerchief)
3,0.146,(Electronic Games)
4,0.144,(Bookcases)
...,...,...
329,0.010,"(T-shirt, Trousers)"
330,0.010,"(Electronic Games, Stole, Accessories)"
331,0.010,"(Hankerchief, Leggings, Accessories)"
332,0.010,"(Hankerchief, Leggings, Shirt)"


## Association Rules

In [83]:
product_assosiation = association_rules(frequent_itemsets,metric='confidence',min_threshold=0.7).sort_values(['support','confidence']).reset_index(drop=True)
product_assosiation

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(Stole, Printers, Shirt)",(Saree),0.014,0.312,0.010,0.714286,2.289377,0.005632,2.408000,0.571197
1,"(Printers, T-shirt, Saree)",(Stole),0.014,0.314,0.010,0.714286,2.274795,0.005604,2.401000,0.568357
2,"(Chairs, Bookcases, Saree)",(Stole),0.014,0.314,0.010,0.714286,2.274795,0.005604,2.401000,0.568357
3,"(Hankerchief, Printers, Electronic Games)",(Stole),0.014,0.314,0.010,0.714286,2.274795,0.005604,2.401000,0.568357
4,"(Printers, T-shirt, Saree)",(Bookcases),0.014,0.144,0.010,0.714286,4.960317,0.007984,2.996000,0.809736
5,"(Stole, Printers, Bookcases)",(T-shirt),0.014,0.140,0.010,0.714286,5.102041,0.008040,3.010000,0.815416
6,"(Hankerchief, Shirt, Accessories)",(Stole),0.014,0.314,0.010,0.714286,2.274795,0.005604,2.401000,0.568357
7,"(Stole, Furnishings, Shirt)",(Hankerchief),0.014,0.276,0.010,0.714286,2.587992,0.006136,2.534000,0.622312
8,"(Hankerchief, Furnishings, Shirt)",(Stole),0.014,0.314,0.010,0.714286,2.274795,0.005604,2.401000,0.568357
9,"(Hankerchief, Shirt, Accessories)",(Saree),0.014,0.312,0.010,0.714286,2.289377,0.005632,2.408000,0.571197
